# Clone Portal users, groups and content

https://developers.arcgis.com/python/sample-notebooks/clone-portal-users-groups-and-content/

This sample notebook can be used for cloning a portal, from say, a staging to a production environment. It clones the users, groups and the content. It does not copy over services though, and works at the tier of portal items.

In [1]:
from arcgis.gis import GIS
from IPython.display import display
from getpass import getpass

In [2]:
source_password = getpass()
target_password = getpass()
source_portal_url = "https://sgctest04.esri.co/portal"
target_portal_url = "https://catastro-sesquile.com/portal"
source_admin_username = 'portaladmin' #'Site4dm1n1920'
source = GIS(source_portal_url, source_admin_username, source_password, verify_cert=False)
target_admin_username = 'portaladmin' #'Sesquile2021'
target = GIS(target_portal_url, target_admin_username, target_password, verify_cert=False)


# Users:

In [ ]:
#!esri_ & !admin
source_users = source.users.search('!esri_ & !admin & !arcgis & !catreg')
for user in source_users:
    print(user.username + "\t:\t" + str(user.role))

In [ ]:
print("Cantidad de usuarios: " + str(len(source_users)))

In [ ]:
# filter out system and initial administrator accounts
target_users = target.users.search('!esri_ & !admin & !system_publisher')
target_users

# Remove existing users from target portal:

In [ ]:
for source_user in source_users:
    try:
        target_user = target.users.get(source_user.username)
        if target_user is not None:
            print('Deleting user: ' + target_user.fullName)
            target_user.reassign_to(target_admin_username)
            target_user.delete()
    except:
        print('User {} does not exist in Target Portal'.format(source_user.username))

# Copy Users:

In [ ]:
def copy_user(target_portal, source_user, password):
    # See if the user has firstName and lastName properties
    try:
        first_name = source_user.firstName
        last_name = source_user.lastName
    except:
        # if not, split the fullName
        full_name = source_user.fullName
        first_name = full_name.split()[0]
        try:
            last_name = full_name.split()[1]
        except:
            last_name = 'NoLastName'

    try:
        print("create user:")
        target_user = target_portal.users.create(source_user.username, password, first_name, 
                                                 last_name, "wmoreno@esri.co",#source_user.email, 
                                                 source_user.description, source_user.role)
        print("user created")

        # update user properties
        print("update user:")
        target_user.update(source_user.access, source_user.preferredView,
                           source_user.description, source_user.tags
                           #,source_user.get_thumbnail_link()
                           #,culture=source_user.culture
                           ,region=source_user.region
                           )
        print("user updated")
        return target_user
    
    except Exception as Ex:
        print(str(Ex))
        print("Unable to create user "+ source_user.username)
        return None

In [ ]:
for user in source_users:
    print("Creating user: " + user.username)
    copy_user(target, user, 'Esrico2021$')

In [ ]:
target_users = target.users.search()
target_users

# Groups:

In [ ]:
# filter out system created groups
source_groups = source.groups.search("!owner:esri_* & !Basemaps")
source_groups

In [ ]:
target_groups = target.groups.search("!owner:esri_* & !Basemaps")
target_groups

In [ ]:
for tg in target_groups:
    for sg in source_groups:
        if sg.title == tg.title and (not tg.owner.startswith('esri_')):
            print("Cleaning up group {} in target Portal...".format(tg.title))
            tg.delete()
            break

# Copy Groups:

In [ ]:
import tempfile

GROUP_COPY_PROPERTIES = ['title', 'description', 'tags', 'snippet', 'phone',
                         'access', 'isInvitationOnly']

def copy_group(target, source, source_group):
    
    with tempfile.TemporaryDirectory() as temp_dir:
        try:
            target_group = {}

            for property_name in GROUP_COPY_PROPERTIES:
                target_group[property_name] = source_group[property_name]

            if source_group['access'] == 'org' and target.properties['portalMode'] == 'singletenant':
                #cloning from ArcGIS Online to ArcGIS Enterprise
                target_group['access'] = 'public'

            elif source_group['access'] == 'public'\
                 and source.properties['portalMode'] == 'singletenant'\
                 and target.properties['portalMode'] == 'multitenant'\
                 and 'id' in target.properties:
                    #cloning from ArcGIS Enterprise to ArcGIS Online org
                    target_group['access'] = 'org'

            # Download the thumbnail (if one exists)
            thumbnail_file = None
            if 'thumbnail' in group:
                target_group['thumbnail'] = group.download_thumbnail(temp_dir)

            # Create the group in the target portal
            copied_group = target.groups.create_from_dict(target_group)

            # Reassign all groups to correct owners, add users, and find shared items
            members = group.get_members()
            if not members['owner'] == target_admin_username:
                copied_group.reassign_to(members['owner'])
            if members['users']:
                copied_group.add_users(members['users'])
            return copied_group
        except:
            print("Error creating " + source_group['title'])

In [ ]:
from IPython.display import display
for group in source_groups:
    target_group = copy_group(target, source, group)
    if target_group:
        display(target_group)

In [ ]:
target_groups = target.groups.search()
target_groups

In [ ]:
group1 = target_groups[0]
group1.get_members()

# Cloning Content
https://developers.arcgis.com/python/guide/cloning-content/


In [ ]:
hosted_flyr = source.content.get("04031d6adac348628c4cac8b48dd436f")
hosted_flyr

In [ ]:
hosted_flyr.url

In [ ]:
cloned_flyr = target.content.clone_items(items=[hosted_flyr]
                                        ,owner="adminsgcv2"
                                        )
cloned_flyr[0]

In [ ]:
cloned_flyr[0].url

In [8]:
# WFM TARGET
itemid_wfm = "bb898a0a6e3444448207d0e13d7ce279"
wfm_content = source.content.get(itemid_wfm)
wfm_content.url


'https://sgctest04.esri.co/server/rest/services/wmx/WorkflowManagerService/WMServer'

In [11]:
from arcgis.features import FeatureLayerCollection
flc = FeatureLayerCollection(wfm_content.url, source)
flc.fromitem

AttributeError: 'PropertyMap' instance has no attribute 'layers'